In [2]:
import numpy as np

In [3]:
class Operation(object):
    '''
    Base class for all operations in a
    neural network.
    
    Created since the other operations classes are
    going to inherit from this class.
    '''
    def __init__(self):
        pass
    
    def forward(self, input_:np.ndarray):
        self.input_ = input_
        self.output = self._output()
        
        return self.output
    
    def backward(self, output_grad:np.ndarray) -> np.ndarray:
        
        assert self.output.shape == output_grad.shape
        self.input_grad = self._input_grad(output_grad) # Check this out
        
        assert self.input_.shape == self.input_grad.shape
        return self.input_grad
    
    def _output(self) -> np.ndarray:
        raise NotImplementedError
    
    def _input_grad(self, output_grad:np.ndarray) -> np.ndarray:
        raise NotImplementedError

In [4]:
class ParamOperation(Operation):
    
    def __init__(self, param:np.ndarray) -> np.ndarray:
        super().__init__()
        self.param = param
        
    def backward(self, output_grad:np.ndarray) -> np.ndarray:
        assert self.output.shape == output_grad.shape
        
        self.input_grad = self._input_grad(output_grad)
        self.param_grad = self._param_grad(output_grad)
        
        assert self.input_ == self.input_grad
        assert self.param == self.param_grad
        
        return self.input_grad

In [6]:
class WeightMultiply(ParamOperation):
    
    def __init__(self, W:np.ndarray) -> np.ndarray:
        super().__init__(W)
        
    def _output(self) -> np.ndarray:
        return np.dot(self.input_, self.param)
    
    def _input_grad(self, output_grad:np.ndarray) -> np.ndarray:
        return np.dot(output_grad, self.param.T)
    
    def _param_grad(self, output_grad:np.ndarray) -> np.ndarray: # Check this
        return np.dot(self.input_.T, output_grad)

In [7]:
class BiasAdd(ParamOperation):
    
    def __init__(self, B:np.ndarray):
        assert B[0].shape == 1
        super().__init__(B)
        
    def _output(self) -> np.ndarray:
        return self.input_ + self.param
    
    def _input_grad(self, output_grad:np.ndarray) -> np.ndarray:
        return np.ones_like(self.input_) * output_grad
    
    def _param_grad(self, output_grad:np.ndarray) -> np.ndarray:
        param_grad = np.ones_like(self.param) * output_grad
        return np.sum(param_grad, axis=0).reshape(1, param_grad.shape[1])

In [9]:
class Sigmoid(Operation):
    
    def __init__(self):
        super().__init__()
        
    def _output(self) -> np.ndarray:
        return 1.0/(1+np.exp(-1.0 * self.input_))
    
    def _input_grad(self, output_grad:np.ndarray) -> np.ndarray:
        sigmoid_backward = self.output * (1.0 - self.output)
        input_grad = sigmoid_backward * output_grad
        return input_grad

In [10]:
class Layer(object):
    
    def __init__(self, neurons: int):
        self.neurons = neurons
        self.first = True
        self.params: list[np.ndarray] = []
        self.param_grads: list[np.ndarray] = []
        self.operations: list[Operation] = []
        
    
    def _setup_layer(self, num_in:int) -> None:
        raise NotImplementedError
    
    def forward(self, input_:np.ndarray) -> np.ndarray:
        
        if self.first:
            self._setup_layer(input_)
            self.first = False
        
        self.input_ = input_
        
        for operation in self.operations:
            input_ = operation.forward(input_)
            
        self.output_ = input_
        
        return self.output_
    
    def backward(self, output_grad:np.ndarray) -> np.ndarray:
        assert self.output.shape == output_grad.shape
        
        for operation in reversed(self.operations):
            output_grad = operation.backward(output_grad)
            
        input_grad = output_grad
        
        self.param_grads()
        
        return input_grad
    
    def param_grads(self) -> np.ndarray:
        self.param_grads = []
        
        for operation in self.operations:
            if issubclass(operation.__class__, ParamOperation):
                self.param_grads.append(operation.param_grad)
                
    def _params(self) -> np.ndarray:
        self.params = []
        
        for operation in self.operations:
            if issubclass(operation.__class__, ParamOperation):
                self.params.append(operation.param)

In [14]:
 class Dense(Layer):
    def __init__(self, neurons: int, activation: Operation = Sigmoid()) -> None:
        super().__init__(neurons)
        self.activation = activation
    
    def _setup_layer(self, input_: np.ndarray) -> None:
        if self.seed:
            np.random.seed(self.seed)
            
        self.params = []
        self.params.append(np.random.randn(input_.shape[1], self.neurons))
        self.params.append(np.random.randn(1, self.neurons))
        
        self.operations = [WeightMultiply(self.params[0]),
                           BiasAdd(self.params[1]),
                           self.activation]
                           
        return None
    
    